In [ ]:
from oct2py import Oct2Py

import os
import pandas as pd
import csv

In [ ]:
V_S = 7.0                   # service speed [kn]

In [ ]:
n_runs = 3 # number of executions 

In [ ]:
seed = 1

Fitness and Evaluation

In [ ]:
# run the Octave script to get the power brake for the params
def run_original(V_S, seed):
    P_B, t075dD,tmin075dD, tal07R,cavLim, Vtip,Vtipmax = [-1, -1, -1, -1, -1, -1, -1]
    output_filename = 'df_' + str(seed) + '.csv'
    with Oct2Py() as octave:
        octave.warning ("off", "Octave:data-file-in-path");
        octave.addpath('./allCodesOctave');
        octave.eval('pkg load statistics')
        D, Z, AEdAO, PdD, P_B = octave.F_DifEvo_LH2_return_constraints(V_S, output_filename, nout=5)
    return [P_B, t075dD,tmin075dD, tal07R,cavLim, Vtip,Vtipmax]

In [ ]:
run_original(V_S, seed)

## Make CSV

In [ ]:
dir_to_save = 'to_csv'

In [ ]:
def get_best_fit_number(x):
    for i in range(len(x)):
        x.iloc[i] = x.iloc[i][len('best fit at iteration '):]
    return x

col_names_original = ['D', 'Z', 'AEdAO', 'PdD', 'P_B', 'n', 'etaO', 'etaR', 't075dD','tmin075dD', 'tal07R','cavLim', 'Vtip','Vtipmax', 'fitness', 'iteration', 'population_i']
col_names = ['D', 'AEdAO', 'PdD', 'Z', 'P_B', 'n', 'fitness', 't075dD','tmin075dD', 'tal07R','cavLim', 'Vtip','Vtipmax', 'penalty', 'valid']

filename = 'df_' + str(seed) + '.csv'

df = pd.read_csv(filename, header=None, skiprows=1, names=col_names_original)

df = df[['D', 'AEdAO', 'PdD', 'Z', 'P_B', 'n', 'fitness', 't075dD','tmin075dD', 'tal07R','cavLim', 'Vtip','Vtipmax']]

# -- Calculate validity --

# insert the other collumns
df.insert(df.shape[1], 'penalty', None)

df.insert(df.shape[1], 'valid', True)

# create temporary
df['temp_cav'] = (df['tal07R'] - df['cavLim'])/df['cavLim']
df['temp_cav'] = df['temp_cav'].clip(lower=0)
df['temp_spd'] = (df['Vtip'] - df['Vtipmax'])/df['Vtipmax']
df['temp_spd'] = df['temp_spd'].clip(lower=0)
df['temp_str'] = (df['tmin075dD'] - df['t075dD'])/df['tmin075dD']
df['temp_str'] = df['temp_str'].clip(lower=0)
# calculate
df['penalty'] = df['temp_cav'] + df['temp_spd'] + df['temp_str']
df['valid'] = (df['penalty'] == 0)
# drop temp
df = df.drop(['temp_cav', 'temp_spd', 'temp_str'], axis=1)


# ajust the iteration counter with fitness
df['D'] = df['D'].astype(str)
df.loc[df['D'].str.contains('best'), 'fitness'] = df.loc[df['D'].str.contains('best'), 'Z']
df.loc[df['D'].str.contains('best'), 'Z'] = None

df.loc[df['D'].str.contains('best'), 'AEdAO'] = get_best_fit_number(df.loc[df['D'].str.contains('best'), 'D'])
df.loc[df['D'].str.contains('best'), 'D'] = 'fitness at iteration'

# -- add best value to the end --
best_fit_to_find = df.loc[len(df)-1]['fitness']
result_row = df[(df['fitness'] == best_fit_to_find) & (df['D'] != 'fitness at iteration')] # find row with best fitness
new_row1 = {'D': 'Best Solution'} # create text row
df.loc[len(df)] = new_row1 # add text row
df.loc[len(df)] = result_row.iloc[0] # add row with values in the end

# Check if the directory already exists
if not os.path.exists(dir_to_save):
    # Create the directory if it doesn't exist
    os.makedirs(dir_to_save)
    print(f"Directory '{dir_to_save}' created.")

filename_out = dir_to_save+'/'+str(seed)+'.csv'
df.to_csv(filename_out, index=False, header=True)

print('saved:', filename_out)
df

# best result file

create file

In [ ]:
dir_name = dir_to_save

# -- functions to save a file and create a dir --
def create_file(text):
    filename = dir_name+'/' + text +'.csv'
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        header = ["D = propeller diameter [m]",
                  "AEdAO = expanded area ratio",
                  "PdD = pitch ratio",
                  "Z = propeller's number of blades",
                  "P_B = power brake",
                  "n = Propeller angular speed [rpm]",
                  "fitness",
                  "i = thread pool id"]
        writer.writerow(header)
    return filename

def append_to_file(filename, row):
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)
        
def append_to_file_order(filename, D='', AEdAO='', PdD='', Z='', P_B='', n='', fitness='', i=''):
    row = [D, AEdAO, PdD, Z, P_B, n, fitness, i]
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

def save_best_result(D, Z, AEdAO, PdD, fitness, seed, solver_name, P_B='', n='', history=''):
    # create the csv file with the headers
    fitness = -fitness
    Z = int(Z)
    filename = create_file('best_results_' + str(seed) + '_' + solver_name)
    append_to_file_order(filename, D=D, AEdAO=AEdAO, PdD=PdD, Z=Z, P_B=P_B, n=P_B, fitness=fitness)
    append_to_file(filename, ['history'])
    append_to_file(filename, history)
    print('saved', filename)

In [ ]:
D       = float(result_row.iloc[0]['D'])
Z       = int(result_row.iloc[0]['Z'])
AEdAO   = float(result_row.iloc[0]['AEdAO'])
PdD     = float(result_row.iloc[0]['PdD'])
fitness = float(result_row.iloc[0]['fitness'])
P_B = float(result_row.iloc[0]['P_B'])
n = float(result_row.iloc[0]['n'])

# history
filename = 'df_' + str(seed) + '.csv'
history = []
with open(filename, 'r', newline='') as file:
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        if 'best' in row[0]:
            # iteration = row[0].split(' ')[-1]
            fit = -float(row[1])
            history.append(str(fit))

save_best_result(D, Z, AEdAO, PdD, fitness, seed, 'original', P_B=P_B, n=n, history=history)